# Title
[]()

In [7]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [8]:

sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict

In [9]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [10]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

# Create test text_dict

In [11]:
test_text1 = """Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p

Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p
 = 0.37) and increased ratings of pain intensity (by 0.7 ± 0.7 cm; P = 0.004; η2p
 = 0.55) and unpleasantness (by 0.7 ± 0.9 cm; P = 0.02; η2p
 = 0.40); these results were not influenced by menstrual phase. Water ingestion reduced thirst perception (visual analog scale, by 2.3 ± 0.9 cm; P < 0.001, η2p
 = 0.88) but did not reduce pain sensitivity. Therefore, hypohydration increases pain sensitivity in women with no influence of menstrual phase.
"""

test_text2 = """Daily Energy Expenditure through the Human Life Course

Total daily energy expenditure (“total expenditure”, MJ/d) reflects daily energy needs and is a critical variable in human health and physiology, yet it is unclear how daily expenditure changes over the life course. Here, we analyze a large, globally diverse database of total expenditure measured by the doubly labeled water method for males and females aged 8 days to 95 yr. We show that total expenditure is strongly related to fat free mass in a power-law manner and identify four distinct metabolic life stages. Fat free mass-adjusted daily expenditure accelerates rapidly in neonates (0-1yr) to ~46% above adult values at ~1 yr, declines slowly throughout childhood and adolescence (1-20 yr) to adult levels at ~20 yr, remains stable in adulthood (20-60 yr) even during pregnancy, and declines in older adults (60+ yr). These changes in total expenditure shed new light on human development and aging and should help shape nutrition and health strategies across the lifespan.
"""

test_text_dict = create_text_dict([test_text1, test_text2])
test_text_dict.keys()

dict_keys([1, 2])

In [15]:
test_text_dict[1]

'Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity i

# Create prompts_df

In [12]:
from itertools import product
prompts_dict = dict()
experiment_num = 1

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

task_part1 = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]
audience = [
    # "lay audience",
    # "",
    "seniors",
    # "people who enjoy sports"
]
task_part2 = [
    "",
    "Use terms a 12-year-old can understand.",
    # "Assume your audience has no science background."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

task_part3 =[
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
    ""
]

prompts_df = pd.DataFrame(product(prep_step, task_part1, task_part2, task_part3, audience), 
    columns=['prep_step', 'task part 1', 'task part 2', 'task part 3', 'audience'])

prompts_df = pd.DataFrame(product(prep_step, task_part1), 
    columns=['prep_step', 'task part 1'])


prompts_df['prompt'] = prompts_df.apply(
    lambda row: f"{row['prep_step']} {row['task part 1']}.", 
    axis=1)
# prompts_df['simplify'] = prompts_df.apply(
#     lambda row: f" {row['task part 2'] if row['task part 2'] else ''}", 
#     axis=1)
# prompts_df['relevance'] = prompts_df.apply(
#     lambda row: f" {row['task part 3']} {row['audience']} " if row['audience'] else '', 
#     axis=1) 
prompts_dict[experiment_num] = prompts_df
prompts_df

,prep_step,task part 1,prompt
0,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post.
1,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.


# Iteration 1

In [13]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() # use an ordered dictionary instead of a regular dictionary
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        print('temp_qna_dict index:', pd.DataFrame(temp_qna_dict[key]).index)
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = [prompt for prompt in simplify_prompts if len(prompt) >1]
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        for summary_key in summary_keys:
            print(f'\tSimplifying {summary_key}...')
            for index, prompt in enumerate(simplify_prompts):
                print(f'\t...Prompt: {prompt}')
                simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
        try:
            temp_qna_chain_dict = pd.concat([qna_chain_dict[summary_iteration_id], pd.DataFrame(simple_summaries_dict)])
            qna_chain_dict[iteration_id] = temp_qna_chain_dict
            print('Simple summaries added to original DataFrame')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Unable to concatenate simple summary DataFrame to original DataFrame')
            qna_chain_dict[iteration_id] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 1
n_choices = 2
qna_dict = dict()
qna_chain_dict = dict()
chatbot_dict = dict()

qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
    iteration_id=iteration_id, n_choices=n_choices)



**Text #1 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
temp_qna_dict index: Index(['article_title', 'system_role', 'prompt', 'model', 'response_01',
       'response_02', 'text'],
      dtype='object')
**Text #2 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
temp_qna_dict index: Index(['article_title', 'system_role', 'prompt', 'model', 'response_01',
       'response_02', 'text'],
      dtype='object')


In [14]:
qna_dict[iteration_id]

,0,1,0,1
article_title,"Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p","Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p",Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"Chronic pain is a major health problem with high costs, but current treatments are often ineffective due to the complex nature of pain. A study found that mild dehydration can increase pain sensitivity in women, with no difference based on menstrual phase. This highlights the importance of hydration for managing pain. #chronicpain #hydration #healthresearch","Hey, did you know that chronic pain is a big problem and current treatments aren't always effective? There's new research suggesting that mild dehydration can actually increase pain sensitivity in women, and it's not affected by the menstrual cycle. The study involved 14 women who were restricted from fluids for 24 hours and they experienced reduced pain tolerance and increased pain intensity and unpleasantness. Water ingestion didn't reduce pain sensitivity, but did reduce thirst perception. These findings could have implications for pain management in women.","New research reveals that daily energy e

In [17]:
task_part2

['', 'Use terms a 12-year-old can understand.']

In [18]:
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
    simplify_iteration=1, summary_iteration_id=None
    )

summary_iteration_id: 1
**text1_prompt00
	Simplifying response_01...
	...Prompt: Use terms a 12-year-old can understand.
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	Simplifying response_02...
	...Prompt: Use terms a 12-year-old can understand.
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
Simple summaries added to original DataFrame
**text1_prompt01
	Simplifying response_01...
	...Prompt: Use terms a 12-year-old can understand.
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done 

In [19]:
qna_dict[iteration_id]

,0,1,0,1
article_title,"Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p","Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p",Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"Chronic pain is a major health problem with high costs, but current treatments are often ineffective due to the complex nature of pain. A study found that mild dehydration can increase pain sensitivity in women, with no difference based on menstrual phase. This highlights the importance of hydration for managing pain. #chronicpain #hydration #healthresearch","Hey, did you know that chronic pain is a big problem and current treatments aren't always effective? There's new research suggesting that mild dehydration can actually increase pain sensitivity in women, and it's not affected by the menstrual cycle. The study involved 14 women who were restricted from fluids for 24 hours and they experienced reduced pain tolerance and increased pain intensity and unpleasantness. Water ingestion didn't reduce pain sensitivity, but did reduce thirst perception. These findings could have implications for pain management in women.","New research reveals that daily energy e

In [23]:
chatbot_dict[1].keys()

dict_keys(['text1_prompt00', 'text1_prompt01', 'text2_prompt00', 'text2_prompt01'])

In [24]:
chatbot_dict[1]['text1_prompt00'].qna

{'article_title': 'Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine 

In [25]:
chatbot_dict[1]['text1_prompt00']

# Update test_text_dict

In [21]:
test_text1 = """Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women 

Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17β-estradiol and progesterone) throughout the menstrual cycle may influence a woman’s pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypohydration. Elevated serum osmolality, plasma copeptin, and urine specific gravity indicated mild hypohydration. Compared with euhydration, HYPO reduced pain tolerance (by 34 ± 46 s; P = 0.02, η2p
 = 0.37) and increased ratings of pain intensity (by 0.7 ± 0.7 cm; P = 0.004; η2p
 = 0.55) and unpleasantness (by 0.7 ± 0.9 cm; P = 0.02; η2p
 = 0.40); these results were not influenced by menstrual phase. Water ingestion reduced thirst perception (visual analog scale, by 2.3 ± 0.9 cm; P < 0.001, η2p
 = 0.88) but did not reduce pain sensitivity. Therefore, hypohydration increases pain sensitivity in women with no influence of menstrual phase.
"""

test_text2 = """Daily Energy Expenditure through the Human Life Course

Total daily energy expenditure (“total expenditure”, MJ/d) reflects daily energy needs and is a critical variable in human health and physiology, yet it is unclear how daily expenditure changes over the life course. Here, we analyze a large, globally diverse database of total expenditure measured by the doubly labeled water method for males and females aged 8 days to 95 yr. We show that total expenditure is strongly related to fat free mass in a power-law manner and identify four distinct metabolic life stages. Fat free mass-adjusted daily expenditure accelerates rapidly in neonates (0-1yr) to ~46% above adult values at ~1 yr, declines slowly throughout childhood and adolescence (1-20 yr) to adult levels at ~20 yr, remains stable in adulthood (20-60 yr) even during pregnancy, and declines in older adults (60+ yr). These changes in total expenditure shed new light on human development and aging and should help shape nutrition and health strategies across the lifespan.
"""

test_text_dict = create_text_dict([test_text1, test_text2])
test_text_dict.keys()

dict_keys([1, 2])

# Iteration 2

In [30]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() # use an ordered dictionary instead of a regular dictionary
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        # print('temp_qna_dict index:', pd.DataFrame(temp_qna_dict[key]).index)
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        for summary_key in summary_keys:
            print(f'\tSimplifying {summary_key}...')
            for index, prompt in enumerate(simplify_prompts):
                print(f'\t...Prompt: {prompt}')
                simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
        try:
            qna_chain_dict[summary_iteration_id].columns = [i for i in range(qna_chain_dict[summary_iteration_id].shape[1])]
            temp_qna_chain_dict = pd.concat([
                qna_chain_dict[summary_iteration_id], 
                pd.DataFrame(simple_summaries_dict, columns=[i for i in range(len(simple_summaries_dict))])])
            qna_chain_dict[iteration_id] = temp_qna_chain_dict
            print('Simple summaries added to original DataFrame')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Unable to concatenate simple summary DataFrame to original DataFrame')
            qna_chain_dict[iteration_id] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 2
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
    iteration_id=iteration_id, n_choices=n_choices)



**Text #1 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!


In [31]:
qna_dict[iteration_id]

,0,1,0,1
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration increases pain sensitivity in women, with no influence of menstrual phase. Compared to euhydration, hypohydration reduced pain tolerance by 34±46s, increased pain intensity by 0.7±0.7cm, and unpleasantness by 0.7±0.9cm. Acute water ingestion did not reduce pain sensitivity but reduced thirst perception. Learn more about the multifactorial nature of pain and potential interactions between hydration and menstrual cycle in this insightful study."" #painperception #hydration #menstrualcycle #healthresearch","Hey, did you know that a recent study found that mild hypohydration, or a state of slight dehydration, can increase pain sensitivity in women? The study involved 14 women and found that hypohydration reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. What's interesting is that this effect was not influenced by menstrual phase. The study also found that acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. It's pretty fascinating to see how hydration levels can affect pain perception in women.","""New research reveals four distinct metabolic life stages impacting daily energy expenditure throughout the human life course. By analyzing a diverse database, it was found that fat-free mass greatly influences energy needs. Daily expenditure accelerates in neonates, declines throughout childhood and adolescence, remains stable in adulthood, and declines in older adults. These findings shed new light on human development and aging, shaping nutrition and health strategies across the lifespan. #humanhealth #nutrition #research""","Hey, did you know that a recent study analyzed a large database of daily energy expenditure in humans from 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat free mass and identified four distinct metabolic life stages. For example, in neonates (0-1 yr), daily expenditure accelerates rapidly to 46% above adult values at 1 year, then declines slowly throughout childhood and adolescence (1-20 yr) before stabilizing at adult levels at around 20 years. Daily expenditure remains stable in adulthood (20-60 yr) even during pregnancy, but declines in older adults (60+ yr). These findings could have important implications for nutrition and health strategies across the lifespan."
response_02,"""New study finds mild hypohydration increases pain sensitivity in women, irrespective of menstrual phase. This research involved 14 eumenorrheic women and showed a reduction in pain tolerance by 34±46 seconds, along with increased ratings of pain intensity and unpleasantness. Acute water ingestion reduced thirst perception but did not reduce pain sensitivity. These findings have important implications for pain management and highlight the multifactorial nature of pain. #ScienceCommunication #PainManagement #Women'sHealth""","Hey, did you know that a recent study found that mild dehydration can increase pain sensitivity in wom

In [32]:
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
    simplify_iteration=1, summary_iteration_id=None
    )
qna_dict[iteration_id]

summary_iteration_id: 2
**text1_prompt00
	Simplifying response_01...
	...Prompt: Use terms a 12-year-old can understand.
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	Simplifying response_02...
	...Prompt: Use terms a 12-year-old can understand.
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
Simple summaries added to original DataFrame
**text1_prompt01
	Simplifying response_01...
	...Prompt: Use terms a 12-year-old can understand.
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done 

,0,1,2,3
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration increases pain sensitivity in women, with no influence of menstrual phase. Compared to euhydration, hypohydration reduced pain tolerance by 34±46s, increased pain intensity by 0.7±0.7cm, and unpleasantness by 0.7±0.9cm. Acute water ingestion did not reduce pain sensitivity but reduced thirst perception. Learn more about the multifactorial nature of pain and potential interactions between hydration and menstrual cycle in this insightful study."" #painperception #hydration #menstrualcycle #healthresearch","Hey, did you know that a recent study found that mild hypohydration, or a state of slight dehydration, can increase pain sensitivity in women? The study involved 14 women and found that hypohydration reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. What's interesting is that this effect was not influenced by menstrual phase. The study also found that acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. It's pretty fascinating to see how hydration levels can affect pain perception in women.","""New research reveals four distinct metabolic life stages impacting daily energy expenditure throughout the human life course. By analyzing a diverse database, it was found that fat-free mass greatly influences energy needs. Daily expenditure accelerates in neonates, declines throughout childhood and adolescence, remains stable in adulthood, and declines in older adults. These findings shed new light on human development and aging, shaping nutrition and health strategies across the lifespan. #humanhealth #nutrition #research""","Hey, did you know that a recent study analyzed a large database of daily energy expenditure in humans from 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat free mass and identified four distinct metabolic life stages. For example, in neonates (0-1 yr), daily expenditure accelerates rapidly to 46% above adult values at 1 year, then declines slowly throughout childhood and adolescence (1-20 yr) before stabilizing at adult levels at around 20 years. Daily expenditure remains stable in adulthood (20-60 yr) even during pregnancy, but declines in older adults (60+ yr). These findings could have important implications for nutrition and health strategies across the lifespan."
response_02,"""New study finds mild hypohydration increases pain sensitivity in women, irrespective of menstrual phase. This research involved 14 eumenorrheic women and showed a reduction in pain tolerance by 34±46 seconds, along with increased ratings of pain intensity and unpleasantness. Acute water ingestion reduced thirst perception but did not reduce pain sensitivity. These findings have important implications for pain management and highlight the multifactorial nature of pain. #ScienceCommunication #PainManagement #Women'sHealth""","Hey, did you know that a recent study found that mild dehydration can increase pain sensitivity in wom

# Iteration 3:
Fix concat issue after simplify prompt
Fix extra API call when simplifying

In [34]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() # use an ordered dictionary instead of a regular dictionary
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
        #     simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
        #     try:
        #         self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
        #         print(f'\t...Summary given')
        #     except:
        #         self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
        #         print(f'\t...Error parsing response for summary request')
        # self.simple_summary_dict['simplify_task'] = simplify_task
        # self.qna = {**self.qna, **self.simple_summary_dict}
        # self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        for summary_key in summary_keys:
            print(f'\tSimplifying {summary_key}...')
            for index, prompt in enumerate(simplify_prompts):
                print(f'\t...Prompt: {prompt}')
                simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
        try:
            qna_chain_dict[summary_iteration_id].columns = [i for i in range(qna_chain_dict[summary_iteration_id].shape[1])]
            temp_qna_chain_dict = pd.concat([
                qna_chain_dict[summary_iteration_id], 
                pd.DataFrame(simple_summaries_dict, columns=[i for i in range(len(simple_summaries_dict))])])
            qna_chain_dict[iteration_id] = temp_qna_chain_dict
            print('Simple summaries added to original DataFrame')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Unable to concatenate simple summary DataFrame to original DataFrame')
            qna_chain_dict[iteration_id] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 3
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
    iteration_id=iteration_id, n_choices=n_choices)



**Text #1 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!


In [35]:
qna_dict[iteration_id]

,0,1,0,1
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study reveals that mild hypohydration can increase pain sensitivity in women, with no influence from menstrual phase. In fact, compared to euhydration, HYPO reduced pain tolerance by 34 ± 46 s and increased ratings of pain intensity and unpleasantness. Acute water ingestion reduced thirst perception, but did not reduce pain sensitivity. These findings have important implications for pain management and treatment. #sciencecommunication #painmanagement #womenhealth""","""Hey, did you know that a recent study found that mild dehydration can increase pain sensitivity in women? They restricted fluid intake for 24 hours and found that it decreased pain tolerance by 34 seconds and increased pain intensity and unpleasantness ratings by 0.7 centimeters. Interestingly, menstrual phase had no effect on these results. They also tried giving the women water to reverse the effects of dehydration, but it only reduced thirst perception, not pain sensitivity. It just goes to show how hydration levels can affect our pain perception.""","""New research reveals the changes in daily energy expenditure throughout the human life course. Analyzing a diverse database, the study identifies four metabolic life stages and shows that fat-free mass strongly influences daily expenditure. Findings show expenditure accelerates rapidly in neonates, declines throughout childhood and adolescence, stabilizes in adulthood (20-60 years), and declines in older adults. These insights shed light on human development and aging and can inform nutrition and health strategies for all ages.""","Hey, did you know that a recent study analyzed a large and diverse database of total energy expenditure in humans of different ages, from 8 days old to 95 years old? They found that total energy expenditure is strongly related to fat-free mass in a power-law manner and identified four distinct metabolic life stages. The study showed that energy expenditure accelerates rapidly in neonates to 46% above adult values at around 1 year, declines slowly throughout childhood and adolescence to adult levels at around 20 years, remains stable in adulthood (20-60 years) even during pregnancy, and declines in older adults (60+ years). These findings shed new light on human development and aging and could inform nutrition and health strategies across the lifespan."
response_02,"New research shows that mild hypohydration increases pain sensitivity in women, but not influenced by menstrual phase. The study found that 24 hours of fluid restriction reduced pain tolerance by 34 seconds and increased pain intensity by 0.7 cm, and unpleasantness by 0.7 cm. Water ingestion reduced thirst perception but did not reduce pain sensitivity. #painperception #hypohydration #menstrualphase #womenshealth #research","Hey, did you know that a new study found that mild dehydration can increase pain sensitivity in women? The study involved 14 women and showed that 24 hours of fluid restriction, which resulted in mild dehydration, reduced pain tolerance by 34 second

In [36]:
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
    simplify_iteration=1, summary_iteration_id=None
    )

summary_iteration_id: 3
**text1_prompt00
	Simplifying response_01...
	...Prompt: Use terms a 12-year-old can understand.
...Preparing to summarize response_01
...Preparing to summarize response_02
	Simplifying response_02...
	...Prompt: Use terms a 12-year-old can understand.
...Preparing to summarize response_01
...Preparing to summarize response_02
Simple summaries added to original DataFrame
**text1_prompt01
	Simplifying response_01...
	...Prompt: Use terms a 12-year-old can understand.
...Preparing to summarize response_01
...Preparing to summarize response_02
	Simplifying response_02...
	...Prompt: Use terms a 12-year-old can understand.
...Preparing to summarize response_01
...Preparing to summarize response_02
Simple summaries added to original DataFrame
**text2_prompt00
	Simplifying response_01...
	...Prompt: Use terms a 12-year-old can understand.
...Preparing to summarize response_01
...Preparing to summarize response_02
	Simplifying response_02...
	...Prompt: Use terms a 12-

## 3.2

In [39]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
        #     simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
        #     try:
        #         self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
        #         print(f'\t...Summary given')
        #     except:
        #         self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
        #         print(f'\t...Error parsing response for summary request')
        # self.simple_summary_dict['simplify_task'] = simplify_task
        # self.qna = {**self.qna, **self.simple_summary_dict}
        # self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        # summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        # for summary_key in summary_keys:
        #     print(f'\tSimplifying {summary_key}...')
        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
        try:
            qna_chain_dict[summary_iteration_id].columns = [i for i in range(qna_chain_dict[summary_iteration_id].shape[1])]
            temp_qna_chain_dict = pd.concat([
                qna_chain_dict[summary_iteration_id], 
                pd.DataFrame(simple_summaries_dict, columns=[i for i in range(len(simple_summaries_dict))])])
            qna_chain_dict[iteration_id] = temp_qna_chain_dict
            print('Simple summaries added to original DataFrame')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Unable to concatenate simple summary DataFrame to original DataFrame')
            qna_chain_dict[iteration_id] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 3
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

# qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
#     iteration_id=iteration_id, n_choices=n_choices)

simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
    simplify_iteration=1, summary_iteration_id=None
    )

summary_iteration_id: 3
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
...Preparing to summarize response_01
...Preparing to summarize response_02
Simple summaries added to original DataFrame
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
...Preparing to summarize response_01
...Preparing to summarize response_02
Simple summaries added to original DataFrame
**text2_prompt00
	...Prompt: Use terms a 12-year-old can understand.
...Preparing to summarize response_01
...Preparing to summarize response_02
Simple summaries added to original DataFrame
**text2_prompt01
	...Prompt: Use terms a 12-year-old can understand.
...Preparing to summarize response_01
...Preparing to summarize response_02
Simple summaries added to original DataFrame


# Iteration 4

In [40]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        # summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        # for summary_key in summary_keys:
        #     print(f'\tSimplifying {summary_key}...')
        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
        try:
            qna_chain_dict[summary_iteration_id].columns = [i for i in range(chaining_bot_dict)]
            temp_qna_chain_dict = pd.concat([
                qna_chain_dict[summary_iteration_id], 
                pd.DataFrame(simple_summaries_dict, columns=[i for i in range(chaining_bot_dict)])])
            qna_chain_dict[iteration_id] = temp_qna_chain_dict
            print('Simple summaries added to original DataFrame')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Unable to concatenate simple summary DataFrame to original DataFrame')
            qna_chain_dict[iteration_id] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 4
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
    iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

**Text #1 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!


In [41]:
qna_dict[iteration_id]

,0,1,0,1
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration, not menstrual phase, increases pain sensitivity in women. In a sample of 14 eumenorrheic women, hypohydration reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. Acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. This study sheds light on the multifactorial nature of pain and the importance of hydration for pain management. #PainSensitivity #Hydration #WomenHealth""","Hey, did you know that mild hypohydration can increase pain sensitivity in women? This study found that 24 hours of fluid restriction reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness by 0.7 cm, regardless of menstrual phase. Interestingly, acute water ingestion didn't reduce pain sensitivity but did reduce thirst perception.","New research shows that total daily energy expenditure changes throughout the human life course, with four distinct metabolic life stages identified. The study analyzed a large, diverse database and found that fat-free mass is strongly related to daily expenditure. In neonates, expenditure accelerates rapidly to 46% above adult values at 1 year, declines throughout childhood and adolescence, remains stable in adulthood (even during pregnancy), and declines in older adults. These findings have important implications for nutrition and health strategies throughout the lifespan. #humanhealth #nutrition #research #energyexpenditure","Hey! Did you know that researchers analyzed a large database of human energy expenditure from people aged 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat-free mass and identified four distinct metabolic life stages. Interestingly, they found that energy expenditure in neonates (0-1 year) accelerates rapidly to 46% above adult values and then slowly declines throughout childhood and adolescence to reach adult levels at 20 years old. Furthermore, energy expenditure remains stable in adulthood (20-60 years old) and declines in older adults (60+ years old). These findings shed new light on human development and aging and could help shape nutrition and health strategies across the lifespan."
response_02,Are you looking for a LinkedIn post that summarizes the key points and numerical descriptors of the given text?,"Hey there! I just read an interesting study that found that mild dehydration can increase pain sensitivity in women. The study involved 14 women who were restricted from fluids for 24 hours, and they found that pain tolerance was reduced by 34 seconds and pain intensity and unpleasantness increased by 0.7 cm. These effects were not influenced by menstrual phase. Interestingly, acute water ingestion did not reduce pain sensitivity, but it did reduce thirst perception. So, it seems that hydration levels may play a role in pain perception, particularly in women.","New research shows that daily energy expenditure changes throughout the human life course. Analyzing a l

In [42]:
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
    simplify_iteration=1, summary_iteration_id=None
    )

summary_iteration_id: 4
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
An error occurred on line 176 in C:\Users\silvh\AppData\Local\Temp\ipykernel_14700\2096265977.py : 'dict' object cannot be interpreted as an integer
Unable to concatenate simple summary DataFrame to original DataFrame
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request t

In [43]:
qna_dict[iteration_id]

{0: {'simple_summary1_01': "Hey there! Did you know that scientists looked at how much energy people use every day? They looked at people from when they were babies to when they were really old. They found that how much energy you use is related to how much muscle you have and that there are four different stages of how our bodies use energy. It's interesting because babies use a lot of energy, more than grown-ups, but then it goes down as they get older. When you're a grown-up, it stays the same, but when you get really old, it goes down again. This information can help us make better choices about what we eat and how we take care of our bodies throughout our lives.",
  'simple_summary1_02': "Hey there! Did you know that scientists did a big study on how much energy people use every day? They looked at people from when they were just born to when they were really old. They found out that how much energy you use is related to how much muscle you have and that there are four different t

## 4.2

In [45]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        # summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        # for summary_key in summary_keys:
        #     print(f'\tSimplifying {summary_key}...')
        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
        try:
            qna_chain_dict[summary_iteration_id].columns = [i for i in range(len(chaining_bot_dict))]
            temp_qna_chain_dict = pd.concat([
                qna_chain_dict[summary_iteration_id], 
                pd.DataFrame(simple_summaries_dict, columns=[i for i in range(len(chaining_bot_dict))])])
            qna_chain_dict[iteration_id] = temp_qna_chain_dict
            print('Simple summaries added to original DataFrame')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Unable to concatenate simple summary DataFrame to original DataFrame')
            # qna_chain_dict[iteration_id] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 4.2
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
    iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

**Text #1 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #1 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #0 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!


In [46]:
qna_dict[iteration_id]

,0,1,0,1
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women'sHealth""","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being well hydrated, hypohydration (24 hours of fluid restriction) reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. This effect was seen in both early follicular and mid-luteal phases of the menstrual cycle. However, acute water ingestion did not reverse the negative effects of dehydration on pain sensitivity. Pretty interesting, right?","New study finds that total daily energy expenditure in humans changes throughout the lifespan, with four distinct metabolic life stages identified. Fat-free mass-adjusted daily expenditure accelerates rapidly in neonates to 46% above adult values at 1 year, declines slowly throughout childhood and adolescence, remains stable in adulthood even during pregnancy, and declines in older adults. These findings shed new light on human development and aging and should inform nutrition and health strategies. #humanhealth #physiology #nutrition #healthstrategies","Hey, did you know that a recent study analyzed a large database of energy expenditure in humans of all ages, from 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat-free mass and identified four distinct metabolic life stages. The first stage is in neonates, where energy expenditure accelerates rapidly to 46% above adult values at 1 year old. Then, energy expenditure declines slowly throughout childhood and adolescence to adult levels at around 20 years old. In adulthood, energy expenditure remains stable even during pregnancy, but it declines in older adults aged 60 and above. These findings shed new light on human development and aging and have important implications for nutrition and health strategies across the lifespan."
response_02,"New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. These findings suggest that hydration may play a role in pain management. #hydration #painsensitivity #womenhealth #research","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being adequately hydrated, a 24-hour fluid restriction reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. Interestingl

In [47]:
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
    simplify_iteration=1, summary_iteration_id=None
    )

summary_iteration_id: 4.2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
Simple summaries added to original DataFrame
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
Simple summaries added to original DataFrame
**text2_prompt00
	...Promp

In [56]:
chatbot_dict[4.2]['text1_prompt00'].qna

{'article_title': 'Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women ',
 'system_role': 'You are an expert at science communication.',
 'prompt': 'Take the key points and numerical descriptors to Summarize for a LinkedIn post.',
 'model': 'gpt-3.5-turbo',
 'response_01': '"New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women\'sHealth"',
 'response_02': 'New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness.

In [59]:
len(chatbot_dict[4.2])

4

In [48]:
qna_dict[iteration_id]

,0,1,2,3
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women'sHealth""","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being well hydrated, hypohydration (24 hours of fluid restriction) reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. This effect was seen in both early follicular and mid-luteal phases of the menstrual cycle. However, acute water ingestion did not reverse the negative effects of dehydration on pain sensitivity. Pretty interesting, right?","New study finds that total daily energy expenditure in humans changes throughout the lifespan, with four distinct metabolic life stages identified. Fat-free mass-adjusted daily expenditure accelerates rapidly in neonates to 46% above adult values at 1 year, declines slowly throughout childhood and adolescence, remains stable in adulthood even during pregnancy, and declines in older adults. These findings shed new light on human development and aging and should inform nutrition and health strategies. #humanhealth #physiology #nutrition #healthstrategies","Hey, did you know that a recent study analyzed a large database of energy expenditure in humans of all ages, from 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat-free mass and identified four distinct metabolic life stages. The first stage is in neonates, where energy expenditure accelerates rapidly to 46% above adult values at 1 year old. Then, energy expenditure declines slowly throughout childhood and adolescence to adult levels at around 20 years old. In adulthood, energy expenditure remains stable even during pregnancy, but it declines in older adults aged 60 and above. These findings shed new light on human development and aging and have important implications for nutrition and health strategies across the lifespan."
response_02,"New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. These findings suggest that hydration may play a role in pain management. #hydration #painsensitivity #womenhealth #research","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being adequately hydrated, a 24-hour fluid restriction reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. Interestingl

## 4.3

In [62]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        # summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        # for summary_key in summary_keys:
        #     print(f'\tSimplifying {summary_key}...')
        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
        try:
            qna_chain_dict[summary_iteration_id].columns = [i for i in range(len(chaining_bot_dict[iteration_id]))]
            temp_qna_chain_dict = pd.concat([
                qna_chain_dict[summary_iteration_id], 
                pd.DataFrame(simple_summaries_dict, columns=[i for i in range(len(chaining_bot_dict[iteration_id]))])])
            qna_chain_dict[iteration_id] = temp_qna_chain_dict
            print('Simple summaries added to original DataFrame')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Unable to concatenate simple summary DataFrame to original DataFrame')
            # qna_chain_dict[iteration_id] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 4.3
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

# qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
#     iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

qna_dict[iteration_id] = qna_dict[4.2].iloc[:10].copy()
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[4.2], iteration_id,
    simplify_iteration=1, summary_iteration_id=4.2
    )

summary_iteration_id: 4.2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
An error occurred on line 176 in C:\Users\silvh\AppData\Local\Temp\ipykernel_14700\2857432783.py : 4.3
Unable to concatenate simple summary DataFrame to original DataFrame
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5

In [63]:
qna_dict[iteration_id]

,0,1,2,3
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women'sHealth""","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being well hydrated, hypohydration (24 hours of fluid restriction) reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. This effect was seen in both early follicular and mid-luteal phases of the menstrual cycle. However, acute water ingestion did not reverse the negative effects of dehydration on pain sensitivity. Pretty interesting, right?","New study finds that total daily energy expenditure in humans changes throughout the lifespan, with four distinct metabolic life stages identified. Fat-free mass-adjusted daily expenditure accelerates rapidly in neonates to 46% above adult values at 1 year, declines slowly throughout childhood and adolescence, remains stable in adulthood even during pregnancy, and declines in older adults. These findings shed new light on human development and aging and should inform nutrition and health strategies. #humanhealth #physiology #nutrition #healthstrategies","Hey, did you know that a recent study analyzed a large database of energy expenditure in humans of all ages, from 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat-free mass and identified four distinct metabolic life stages. The first stage is in neonates, where energy expenditure accelerates rapidly to 46% above adult values at 1 year old. Then, energy expenditure declines slowly throughout childhood and adolescence to adult levels at around 20 years old. In adulthood, energy expenditure remains stable even during pregnancy, but it declines in older adults aged 60 and above. These findings shed new light on human development and aging and have important implications for nutrition and health strategies across the lifespan."
response_02,"New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. These findings suggest that hydration may play a role in pain management. #hydration #painsensitivity #womenhealth #research","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being adequately hydrated, a 24-hour fluid restriction reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. Interestingl

## 4.4

In [64]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        # summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        # for summary_key in summary_keys:
        #     print(f'\tSimplifying {summary_key}...')
        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
    try:
        qna_chain_dict[summary_iteration_id].columns = [i for i in range(len(chaining_bot_dict[iteration_id]))]
        temp_qna_chain_dict = pd.concat([
            qna_chain_dict[summary_iteration_id], 
            pd.DataFrame(simple_summaries_dict, columns=[i for i in range(len(chaining_bot_dict[iteration_id]))])])
        qna_chain_dict[iteration_id] = temp_qna_chain_dict
        print('Simple summaries added to original DataFrame')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        print('Unable to concatenate simple summary DataFrame to original DataFrame')
        # qna_chain_dict[iteration_id] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 4.4
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

# qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
#     iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

qna_dict[iteration_id] = qna_dict[4.2].iloc[:10].copy()
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[4.2], iteration_id,
    simplify_iteration=1, summary_iteration_id=4.2
    )

summary_iteration_id: 4.2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text2_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done c

## 4.5

In [66]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        # summary_keys = [key for key in chaining_bot_dict[text_prompt_key].qna if 'response' in key]

        # for summary_key in summary_keys:
        #     print(f'\tSimplifying {summary_key}...')
        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[index] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
    try:
        qna_chain_dict[summary_iteration_id].columns = [i for i in range(len(chaining_bot_dict[iteration_id]))]
        temp_qna_chain_dict = pd.concat([
            qna_chain_dict[summary_iteration_id], 
            pd.DataFrame(simple_summaries_dict, columns=[i for i in range(len(chaining_bot_dict[iteration_id]))])])
        qna_chain_dict[iteration_id] = temp_qna_chain_dict
        print('Simple summaries added to original DataFrame')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        print('Unable to concatenate simple summary DataFrame to original DataFrame')
        qna_chain_dict[iteration_id+0.001] = pd.DataFrame(simple_summaries_dict)

    return qna_chain_dict



iteration_id = 4.5
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

# qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
#     iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

qna_dict[iteration_id] = qna_dict[4.2].iloc[:10].copy()
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[4.2], iteration_id,
    simplify_iteration=1, summary_iteration_id=4.2
    )

summary_iteration_id: 4.2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text2_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done c

In [68]:
qna_dict[iteration_id+0.001]

,0
simple_summary1_01,"Hey, did you know that scientists studied how much energy people use every day? They looked at people from when they were babies to when they were really old. They found that how much energy you use is related to how much muscle you have and that there are four different stages of how much energy you use in your life. When you're a baby, you use a lot of energy, but then it goes down as you grow up. When you're an adult, it stays about the same, but when you get really old, it goes down again. This is important for knowing how to eat healthy and stay healthy throughout your life."
simple_summary1_02,"Hey, did you know that scientists studied how much energy people need every day as they grow up and get older? They found that babies need a lot of energy, but as we get older, we need less and less until we're grown-ups. Grown-ups need about the same amount of energy every day, even when they're having a baby. But when people get really old, they need less energy again. This information can help us make good choices about what we eat and how we take care of our bodies as we grow up and get older."
simplify_task,Use terms a 12-year-old can understand.


## 4.6

In [70]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for text_prompt_key in chaining_bot_dict:
        print(f'**{text_prompt_key}')
        simple_summaries_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[text_prompt_key][index] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
    try:
        qna_chain_dict[summary_iteration_id].columns = [i for i in range(len(chaining_bot_dict[iteration_id]))]
        temp_qna_chain_dict = pd.concat([
            qna_chain_dict[summary_iteration_id], 
            pd.DataFrame(simple_summaries_dict, columns=[i for i in range(len(chaining_bot_dict[iteration_id]))])])
        qna_chain_dict[iteration_id] = temp_qna_chain_dict
        print('Simple summaries added to original DataFrame')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        print('Unable to concatenate simple summary DataFrame to original DataFrame')
        qna_chain_dict[iteration_id+0.001] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 4.6
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

# qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
#     iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

qna_dict[iteration_id] = qna_dict[4.2].iloc[:10].copy()
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[4.2], iteration_id,
    simplify_iteration=1, summary_iteration_id=4.2
    )

summary_iteration_id: 4.2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text2_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done c

In [79]:
print(len(qna_dict[iteration_id+0.001]))
qna_dict[iteration_id+0.001]

4


{'text1_prompt00': {0: {'simple_summary1_01': "A new study found that when women don't drink enough water, they feel more pain, even if they are not on their period. The study looked at 14 women who have regular periods and found that when they didn't drink enough water for 24 hours, they could tolerate pain for 34 seconds less and felt more pain. Drinking water after this didn't help reduce the pain. So, it's important for women to drink enough water to avoid feeling more pain.",
   'simple_summary1_02': "A new study found that when women don't drink enough water, they feel more pain. This is true no matter what time of the month it is. The study looked at 14 women who have regular periods. They were asked to drink less water for a day. After that, they felt more pain and it was harder to tolerate. Drinking water didn't make the pain go away, but it did make them less thirsty. This means that drinking enough water is important for managing pain.",
   'simplify_task': 'Use terms a 12-y

In [82]:
pd.concat(
    [pd.DataFrame(value) for key, value in qna_dict[iteration_id+0.001].items()], 
    axis=1
    )

,0,0,0,0
simple_summary1_01,"A new study found that when women don't drink enough water, they feel more pain, even if they are not on their period. The study looked at 14 women who have regular periods and found that when they didn't drink enough water for 24 hours, they could tolerate pain for 34 seconds less and felt more pain. Drinking water after this didn't help reduce the pain. So, it's important for women to drink enough water to avoid feeling more pain.","Hey, did you know that not drinking enough water can make women feel more pain? A study found that when women didn't drink enough water for a day, they could handle pain for 34 seconds less and felt more pain. This happened during different times of the month when women's bodies are changing. Drinking water right away didn't make the pain go away. It's important to drink enough water to avoid feeling more pain.","Scientists have found that the amount of energy our bodies use changes as we grow up and get older. They identified four different stages of life where our bodies use energy differently. When we are babies, we use a lot of energy compared to adults. As we grow up, we use less energy until we become adults. Even when women are pregnant, their energy use stays the same. But as we get older, we start to use less energy again. This information can help us make better choices about what we eat and how we take care of our bodies.","Hey, did you know that scientists studied how much energy people use every day? They looked at people from when they were babies to when they were really old. They found that how much energy you use is related to how much muscle you have and that there are four different stages of how much energy you use in your life. When you're a baby, you use a lot of energy, but then it goes down as you grow up. When you're an adult, it stays about the same, but when you get really old, it goes down again. This is important for knowing how to eat healthy and stay healthy throughout your life."
simple_summary1_02,"A new study found that when women don't drink enough water, they feel more pain. This is true no matter what time of the month it is. The study looked at 14 women who have regular periods. They were asked to drink less water for a day. After that, they felt more pain and it was harder to tolerate. Drinking water didn't make the pain go away, but it did make them less thirsty. This means that drinking enough water is important for managing pain.","Hey, did you know that not drinking enough water can make women feel more pain? A study found that when women didn't drink enough water for a whole day, they could only handle pain for 34 seconds less than when they were drinking enough water. They also said the pain felt worse and was more unpleasant. This happened no matter what time of the month it was. Drinking water didn't make the pain go away, but it did make them less thirsty. So, it's really important for women to drink enough water to help manage their pain.","Scientists have found out that the amount of energy our bodies use changes as we grow up and get older. They looked at a lot of information about people from when they were just born to when they were 95 years old. They found that there are four different times in our lives when our bodies use energy in different ways. When we are babies, we use a lot of energy, but as we get older, we use less. This information can help us make better choices about what we eat and how we take care of our bodies as we grow up.","Hey, did you know that scientists studied how much energy people need every day as they grow up and get older? They found that babies need a lot of energy, but as we get older, we need less and less until we're grown-ups. Grown-ups need about the same amount of energy every day, even when they're having a baby. But when people get really old, they need less energy again. This information can help us make good choices about what we eat and how we take care of our bodies as we 

In [72]:
pd.DataFrame(qna_dict[iteration_id+0.001])

,text1_prompt00,text1_prompt01,text2_prompt00,text2_prompt01
0,"{'simple_summary1_01': 'A new study found that when women don't drink enough water, they feel more pain, even if they are not on their period. The study looked at 14 women who have regular periods and found that when they didn't drink enough water for 24 hours, they could tolerate pain for 34 seconds less and felt more pain. Drinking water after this didn't help reduce the pain. So, it's important for women to drink enough water to avoid feeling more pain.', 'simple_summary1_02': 'A new study found that when women don't drink enough water, they feel more pain. This is true no matter what time of the month it is. The study looked at 14 women who have regular periods. They were asked to drink less water for a day. After that, they felt more pain and it was harder to tolerate. Drinking water didn't make the pain go away, but it did make them less thirsty. This means that drinking enough water is important for managing pain.', 'simplify_task': 'Use terms a 12-year-old can understand.'}","{'simple_summary1_01': 'Hey, did you know that not drinking enough water can make women feel more pain? A study found that when women didn't drink enough water for a day, they could handle pain for 34 seconds less and felt more pain. This happened during different times of the month when women's bodies are changing. Drinking water right away didn't make the pain go away. It's important to drink enough water to avoid feeling more pain.', 'simple_summary1_02': 'Hey, did you know that not drinking enough water can make women feel more pain? A study found that when women didn't drink enough water for a whole day, they could only handle pain for 34 seconds less than when they were drinking enough water. They also said the pain felt worse and was more unpleasant. This happened no matter what time of the month it was. Drinking water didn't make the pain go away, but it did make them less thirsty. So, it's really important for women to drink enough water to help manage their pain.', 'simplify_task': 'Use terms a 12-year-old can understand.'}","{'simple_summary1_01': 'Scientists have found that the amount of energy our bodies use changes as we grow up and get older. They identified four different stages of life where our bodies use energy differently. When we are babies, we use a lot of energy compared to adults. As we grow up, we use less energy until we become adults. Even when women are pregnant, their energy use stays the same. But as we get older, we start to use less energy again. This information can help us make better choices about what we eat and how we take care of our bodies.', 'simple_summary1_02': 'Scientists have found out that the amount of energy our bodies use changes as we grow up and get older. They looked at a lot of information about people from when they were just born to when they were 95 years old. They found that there are four different times in our lives when our bodies use energy in different ways. When we are babies, we use a lot of energy, but as we get older, we use less. This information can help us make better choices about what we eat and how we take care of our bodies as we grow up.', 'simplify_task': 'Use terms a 12-year-old can understand.'}","{'simple_summary1_01': 'Hey, did you know that scientists studied how much energy people use every day? They looked at people from when they were babies to when they were really old. They found that how much energy you use is related to how much muscle you have and that there are four different stages of how much energy you use in your life. When you're a baby, you use a lot of energy, but then it goes down as you grow up. When you're an adult, it stays about the same, but when you get really old, it goes down again. This is important for knowing how to eat healthy and stay healthy throughout your life.', 'simple_summary1_02': 'Hey, did you know that scientists studied how much energy people need every day as they gr

## 4.7

In [86]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for column, text_prompt_key in enumerate(chaining_bot_dict):
        print(f'**{text_prompt_key}')
        simple_summaries_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[text_prompt_key][column] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
    try:
        qna_chain_dict[summary_iteration_id].columns = [i for i in range(len(chaining_bot_dict[iteration_id]))]
        simple_summaries_df = pd.concat([pd.DataFrame(value) for key, value in simple_summaries_dict.items()],axis=1)
        simple_summaries_df.columns = [i for i in range(simple_summaries_df.shape[1])]
        temp_qna_chain_dict = pd.concat([
            qna_chain_dict[summary_iteration_id], 
            simple_summaries_df
            ])
        qna_chain_dict[iteration_id] = temp_qna_chain_dict
        print('Simple summaries added to original DataFrame')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        print('Unable to concatenate simple summary DataFrame to original DataFrame')
        qna_chain_dict[iteration_id+0.001] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 4.7
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

# qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
#     iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

qna_dict[iteration_id] = qna_dict[4.2].iloc[:10].copy()
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[4.2], iteration_id,
    simplify_iteration=1, summary_iteration_id=4.2
    )

summary_iteration_id: 4.2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text2_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done c

In [88]:
qna_dict[iteration_id+0.001]

{'text1_prompt00': {0: {'simple_summary1_01': "A new study found that when women don't drink enough water, they feel more pain, even if they are not on their period. The study looked at 14 women who have regular periods and found that when they didn't drink enough water for 24 hours, they could tolerate pain for 34 seconds less and felt more pain. Drinking water after this didn't help reduce the pain. This study is important for understanding how important it is to drink enough water for our health.",
   'simple_summary1_02': "A new study found that when women don't drink enough water, they feel more pain. This is true no matter what time of the month it is. The study looked at 14 women who have regular periods. They were asked to drink less water for a day. After that, they felt more pain and it was harder to tolerate. Drinking water didn't make the pain go away, but it did make them less thirsty. This means that drinking enough water is important for managing pain.",
   'simplify_tas

## 4.8

In [89]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for column, text_prompt_key in enumerate(chaining_bot_dict):
        print(f'**{text_prompt_key}')
        simple_summaries_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[text_prompt_key][column] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
    try:
        qna_chain_dict[summary_iteration_id].columns = [i for i in range(len(chaining_bot_dict[summary_iteration_id]))]
        simple_summaries_df = pd.concat([pd.DataFrame(value) for key, value in simple_summaries_dict.items()],axis=1)
        simple_summaries_df.columns = [i for i in range(simple_summaries_df.shape[1])]
        temp_qna_chain_dict = pd.concat([
            qna_chain_dict[summary_iteration_id], 
            simple_summaries_df
            ])
        qna_chain_dict[iteration_id] = temp_qna_chain_dict
        print('Simple summaries added to original DataFrame')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        print('Unable to concatenate simple summary DataFrame to original DataFrame')
        qna_chain_dict[iteration_id+0.001] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 4.8
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

# qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
#     iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

qna_dict[iteration_id] = qna_dict[4.2].iloc[:10].copy()
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[4.2], iteration_id,
    simplify_iteration=1, summary_iteration_id=4.2
    )

summary_iteration_id: 4.2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text2_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done c

In [91]:
qna_dict[iteration_id+0.001]

{'text1_prompt00': {0: {'simple_summary1_01': "A new study found that when women don't drink enough water, they feel more pain, even if they are not on their period. The study looked at 14 women who have regular periods and found that when they didn't drink enough water for 24 hours, they could tolerate pain for 34 seconds less and felt more pain. Drinking water after this didn't help reduce the pain. So, it's important for women to drink enough water to avoid feeling more pain.",
   'simple_summary1_02': "A new study found that when women don't drink enough water, they feel more pain. This is true no matter what time of the month it is. The study looked at 14 women who have regular periods. They were asked to drink less water for a day. After that, they felt more pain and it was harder to tolerate. Drinking water didn't make the pain go away, but it did make them less thirsty. This means that drinking enough water is important for managing pain.",
   'simplify_task': 'Use terms a 12-y

In [90]:
qna_dict[iteration_id]

,0,1,2,3
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women'sHealth""","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being well hydrated, hypohydration (24 hours of fluid restriction) reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. This effect was seen in both early follicular and mid-luteal phases of the menstrual cycle. However, acute water ingestion did not reverse the negative effects of dehydration on pain sensitivity. Pretty interesting, right?","New study finds that total daily energy expenditure in humans changes throughout the lifespan, with four distinct metabolic life stages identified. Fat-free mass-adjusted daily expenditure accelerates rapidly in neonates to 46% above adult values at 1 year, declines slowly throughout childhood and adolescence, remains stable in adulthood even during pregnancy, and declines in older adults. These findings shed new light on human development and aging and should inform nutrition and health strategies. #humanhealth #physiology #nutrition #healthstrategies","Hey, did you know that a recent study analyzed a large database of energy expenditure in humans of all ages, from 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat-free mass and identified four distinct metabolic life stages. The first stage is in neonates, where energy expenditure accelerates rapidly to 46% above adult values at 1 year old. Then, energy expenditure declines slowly throughout childhood and adolescence to adult levels at around 20 years old. In adulthood, energy expenditure remains stable even during pregnancy, but it declines in older adults aged 60 and above. These findings shed new light on human development and aging and have important implications for nutrition and health strategies across the lifespan."
response_02,"New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. These findings suggest that hydration may play a role in pain management. #hydration #painsensitivity #womenhealth #research","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being adequately hydrated, a 24-hour fluid restriction reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. Interestingl

## 4.9

In [92]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def simplify_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):

    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for column, text_prompt_key in enumerate(chaining_bot_dict):
        print(f'**{text_prompt_key}')
        simple_summaries_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[text_prompt_key][column] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
    try:
        qna_chain_dict[summary_iteration_id].columns = [i for i in range(len(qna_chain_dict[summary_iteration_id].columns))]
        simple_summaries_df = pd.concat([pd.DataFrame(value) for key, value in simple_summaries_dict.items()],axis=1)
        simple_summaries_df.columns = [i for i in range(simple_summaries_df.shape[1])]
        temp_qna_chain_dict = pd.concat([
            qna_chain_dict[summary_iteration_id], 
            simple_summaries_df
            ])
        qna_chain_dict[iteration_id] = temp_qna_chain_dict
        print('Simple summaries added to original DataFrame')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        print('Unable to concatenate simple summary DataFrame to original DataFrame')
        qna_chain_dict[iteration_id+0.001] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 4.9
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

# qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
#     iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

qna_dict[iteration_id] = qna_dict[4.2].iloc[:10].copy()
simplify_prompts = task_part2
qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[4.2], iteration_id,
    simplify_iteration=1, summary_iteration_id=4.2
    )

summary_iteration_id: 4.2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text2_prompt00
	...Prompt: Use terms a 12-year-old can understand.
		...Preparing to summarize response_01
	Done c

### Got it

In [93]:
qna_dict[iteration_id]

,0,1,2,3
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women'sHealth""","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being well hydrated, hypohydration (24 hours of fluid restriction) reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. This effect was seen in both early follicular and mid-luteal phases of the menstrual cycle. However, acute water ingestion did not reverse the negative effects of dehydration on pain sensitivity. Pretty interesting, right?","New study finds that total daily energy expenditure in humans changes throughout the lifespan, with four distinct metabolic life stages identified. Fat-free mass-adjusted daily expenditure accelerates rapidly in neonates to 46% above adult values at 1 year, declines slowly throughout childhood and adolescence, remains stable in adulthood even during pregnancy, and declines in older adults. These findings shed new light on human development and aging and should inform nutrition and health strategies. #humanhealth #physiology #nutrition #healthstrategies","Hey, did you know that a recent study analyzed a large database of energy expenditure in humans of all ages, from 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat-free mass and identified four distinct metabolic life stages. The first stage is in neonates, where energy expenditure accelerates rapidly to 46% above adult values at 1 year old. Then, energy expenditure declines slowly throughout childhood and adolescence to adult levels at around 20 years old. In adulthood, energy expenditure remains stable even during pregnancy, but it declines in older adults aged 60 and above. These findings shed new light on human development and aging and have important implications for nutrition and health strategies across the lifespan."
response_02,"New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. These findings suggest that hydration may play a role in pain management. #hydration #painsensitivity #womenhealth #research","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being adequately hydrated, a 24-hour fluid restriction reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. Interestingl

# Add context

## Update prompts_df

In [112]:
from itertools import product
prompts_dict = dict()
experiment_num = 1

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

task_part1 = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]
audience = [
    # "lay audience",
    # "",
    "seniors",
    # "people who enjoy sports"
]
task_part2 = [
    "",
    "Use terms a 12-year-old can understand.",
    # "Assume your audience has no science background."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

task_part3 =[
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
    ""
]

prompts_df = pd.DataFrame(product(prep_step, task_part1, task_part2, task_part3, audience), 
    columns=['prep_step', 'task part 1', 'task part 2', 'task part 3', 'audience'])

# prompts_df = pd.DataFrame(product(prep_step, task_part1), 
#     columns=['prep_step', 'task part 1'])


prompts_df['prompt'] = prompts_df.apply(
    lambda row: f"{row['prep_step']} {row['task part 1']}.", 
    axis=1)
# prompts_df['simplify'] = prompts_df.apply(
#     lambda row: f" {row['task part 2'] if row['task part 2'] else ''}", 
#     axis=1)
prompts_df['relevance'] = prompts_df.apply(
    lambda row: f" {row['task part 3']} {row['audience']} " if row['audience'] else '', 
    axis=1) 
prompts_dict[experiment_num] = prompts_df
prompts_df

,prep_step,task part 1,task part 2,task part 3,audience,prompt,relevance
0,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,,Add 1-2 sentences to make this relevant for,seniors,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Add 1-2 sentences to make this relevant for seniors
1,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Use terms a 12-year-old can understand.,Add 1-2 sentences to make this relevant for,seniors,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Add 1-2 sentences to make this relevant for seniors
2,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,,Add 1-2 sentences to make this relevant for,seniors,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Add 1-2 sentences to make this relevant for seniors
3,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Use terms a 12-year-old can understand.,Add 1-2 sentences to make this relevant for,seniors,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Add 1-2 sentences to make this relevant for seniors


In [113]:
prompts_df['relevance'].unique()

array([' Add 1-2 sentences to make this relevant for seniors '],
      dtype=object)

In [124]:
import traceback
from collections import OrderedDict

class Chaining:

    def __init__(self, text):
        self.text = text

    def summarize(self, task, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chatbot class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chatbot(self.text,
            system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
            max_tokens=max_tokens
            )
        prompt = chatbot.create_prompt(task)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = system_role
        self.qna['prompt'] = task
        self.qna['model'] = model
        self.summaries_dict = dict()
        self.article_title = title

        try:
            response = chatbot.gpt(prompt)
        except:
            print('\t**API request failed**')
            return self.qna, chatbot
        try:
            for index, choice in enumerate(response.choices):
                self.qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna[f'text'] = self.text
            return self.qna
        except:
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None
                    ):
        self.simplify_bot_dict = dict()
        self.simple_summary_dict = dict()
        for key, summary in self.summaries_dict.items():
            new_key = re.sub(r'response', rf'simple_summary{str(simplify_iteration)}', key)
            print(f'\t\t...Preparing to summarize {key}')
            self.simplify_bot_dict[new_key] = Chatbot(
                summary, system_role=system_role,
                model=model, temperature=temperature, n_choices=n_choices, max_tokens=max_tokens
                )
            simplify_prompt = self.simplify_bot_dict[new_key].create_prompt(simplify_task)
            try:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt).choices[0]["message"]["content"]
                print(f'\t...Summary given')
            except:
                self.simple_summary_dict[new_key] = self.simplify_bot_dict[new_key].gpt(simplify_prompt)
                print(f'\t...Error parsing response for summary request')
        self.simple_summary_dict['simplify_task'] = simplify_task
        self.qna = {**self.qna, **self.simple_summary_dict}
        self.qna.update({'text': self.qna.pop('text')})
        return self.simple_summary_dict

def batch_summarize_chain(text_dict, prompts_df, qna_dict,  chatbot_dict, iteration_id, n_choices=5,
    prompt_column='prompt',
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prompts_df: DataFrame containing the prompts.
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    prompts_df = prompts_df.reset_index(drop=True)
    qna_dfs_list = []
    chatbot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = OrderedDict()
        for index, input in prompts_df[prompt_column].items():
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            try:
                chatbot = Chaining(text)
                temp_qna_dict[key][index] = chatbot.summarize(input, n_choices=n_choices)
                chatbot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
            except:
                print('\t...Error making chatbot request')
                break
        try:
            prompts_df = prompts_df.drop([prompt_column, 'system_role'])
        except:
            pass
        try:
            updated_qna_dict = pd.concat([
                pd.DataFrame(temp_qna_dict[key]),
                prompts_df.transpose()
            ])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chatbot_dict
        try:
            qna_dfs_list.append(updated_qna_dict)
            qna_dict[iteration_id] = pd.concat(qna_dfs_list, axis=1)
            if save_outputs:
                try:
                    append_version = append_version if filename else False
                    filename = filename if filename else f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_prompt_experiments_{"{:02d}".format(iteration_id)}'
                    savepickle(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                    save_csv(qna_dict[iteration_id], filename=filename, path=pickle_path, append_version=append_version)
                except:
                    print('Unable to save outputs')
        except:
            print('Error concatenating DataFrames')
    return qna_dict, chatbot_dict

def prompt_chaining_dict(simplify_prompts, qna_chain_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None,
    prompt_column='simplify', simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    simple_summaries_dict = dict()
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    for column, text_prompt_key in enumerate(chaining_bot_dict):
        print(f'**{text_prompt_key}')
        simple_summaries_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            print(f'\t...Prompt: {prompt}')
            simple_summaries_dict[text_prompt_key][column] = chaining_bot_dict[text_prompt_key].simplify(
                prompt, 
                simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                )
    try:
        qna_chain_dict[summary_iteration_id].columns = [i for i in range(len(qna_chain_dict[summary_iteration_id].columns))]
        simple_summaries_df = pd.concat([pd.DataFrame(value) for key, value in simple_summaries_dict.items()],axis=1)
        simple_summaries_df.columns = [i for i in range(simple_summaries_df.shape[1])]
        temp_qna_chain_dict = pd.concat([
            qna_chain_dict[summary_iteration_id], 
            simple_summaries_df
            ])
        qna_chain_dict[iteration_id] = temp_qna_chain_dict
        print(f"{'Simple summaries' if prompt_column=='simplify' else 'Article relevance for user'} added to original DataFrame")
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        print('Unable to concatenate simple summary DataFrame to original DataFrame')
        qna_chain_dict[iteration_id+0.001] = simple_summaries_dict

    return qna_chain_dict



iteration_id = 5
n_choices = 2
# qna_dict = dict()
# qna_chain_dict = dict()
# chatbot_dict = dict()

# qna_dict, chaining_dict = batch_summarize_chain(test_text_dict, prompts_df, qna_dict, chatbot_dict, 
#     iteration_id=iteration_id, n_choices=n_choices)

# simplify_prompts = task_part2
# qna_dict = simplify_chaining_dict(simplify_prompts, qna_dict, chaining_dict[iteration_id], iteration_id,
#     simplify_iteration=1, summary_iteration_id=None
#     )

summary_iteration_id = 4.2
qna_dict[iteration_id] = qna_dict[4.2].iloc[:10].copy()
simplify_prompts = task_part2
relevance_prompts = prompts_df['relevance']
qna_dict = prompt_chaining_dict(relevance_prompts, qna_dict, chaining_dict[summary_iteration_id], iteration_id,
    prompt_column='relevance',
    simplify_iteration=2, summary_iteration_id=summary_iteration_id
    )

summary_iteration_id: 4.2
**text1_prompt00
	...Prompt:  Add 1-2 sentences to make this relevant for seniors 
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text1_prompt01
	...Prompt:  Add 1-2 sentences to make this relevant for seniors 
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
**text2_prompt00
	...Prompt:  Add 1-2 sentences to make this relevant for seniors 
		...

In [125]:
qna_dict[iteration_id]

,0,1,2,3
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women'sHealth""","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being well hydrated, hypohydration (24 hours of fluid restriction) reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. This effect was seen in both early follicular and mid-luteal phases of the menstrual cycle. However, acute water ingestion did not reverse the negative effects of dehydration on pain sensitivity. Pretty interesting, right?","New study finds that total daily energy expenditure in humans changes throughout the lifespan, with four distinct metabolic life stages identified. Fat-free mass-adjusted daily expenditure accelerates rapidly in neonates to 46% above adult values at 1 year, declines slowly throughout childhood and adolescence, remains stable in adulthood even during pregnancy, and declines in older adults. These findings shed new light on human development and aging and should inform nutrition and health strategies. #humanhealth #physiology #nutrition #healthstrategies","Hey, did you know that a recent study analyzed a large database of energy expenditure in humans of all ages, from 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat-free mass and identified four distinct metabolic life stages. The first stage is in neonates, where energy expenditure accelerates rapidly to 46% above adult values at 1 year old. Then, energy expenditure declines slowly throughout childhood and adolescence to adult levels at around 20 years old. In adulthood, energy expenditure remains stable even during pregnancy, but it declines in older adults aged 60 and above. These findings shed new light on human development and aging and have important implications for nutrition and health strategies across the lifespan."
response_02,"New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. These findings suggest that hydration may play a role in pain management. #hydration #painsensitivity #womenhealth #research","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being adequately hydrated, a 24-hour fluid restriction reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. Interestingl

In [120]:
qna_dict[4.2] = qna_dict[4.2].iloc[:10]

In [121]:
qna_dict[4.2]

,0,1,2,3
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Daily Energy Expenditure through the Human Life Course,Daily Energy Expenditure through the Human Life Course
system_role,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.,You are an expert at science communication.
prompt,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.
model,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"""New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women'sHealth""","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being well hydrated, hypohydration (24 hours of fluid restriction) reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. This effect was seen in both early follicular and mid-luteal phases of the menstrual cycle. However, acute water ingestion did not reverse the negative effects of dehydration on pain sensitivity. Pretty interesting, right?","New study finds that total daily energy expenditure in humans changes throughout the lifespan, with four distinct metabolic life stages identified. Fat-free mass-adjusted daily expenditure accelerates rapidly in neonates to 46% above adult values at 1 year, declines slowly throughout childhood and adolescence, remains stable in adulthood even during pregnancy, and declines in older adults. These findings shed new light on human development and aging and should inform nutrition and health strategies. #humanhealth #physiology #nutrition #healthstrategies","Hey, did you know that a recent study analyzed a large database of energy expenditure in humans of all ages, from 8 days to 95 years old? They found that daily energy expenditure is strongly related to fat-free mass and identified four distinct metabolic life stages. The first stage is in neonates, where energy expenditure accelerates rapidly to 46% above adult values at 1 year old. Then, energy expenditure declines slowly throughout childhood and adolescence to adult levels at around 20 years old. In adulthood, energy expenditure remains stable even during pregnancy, but it declines in older adults aged 60 and above. These findings shed new light on human development and aging and have important implications for nutrition and health strategies across the lifespan."
response_02,"New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reduce pain sensitivity but did reduce thirst perception. These findings suggest that hydration may play a role in pain management. #hydration #painsensitivity #womenhealth #research","Hey, did you know that mild dehydration can increase pain sensitivity in women? A recent study found that compared to being adequately hydrated, a 24-hour fluid restriction reduced pain tolerance by 34 seconds and increased ratings of pain intensity and unpleasantness. Interestingl

In [106]:
chatbot_dict[4.2][list(chatbot_dict[4.2].keys())[0]].qna

{'article_title': 'Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women ',
 'system_role': 'You are an expert at science communication.',
 'prompt': 'Take the key points and numerical descriptors to Summarize for a LinkedIn post.',
 'model': 'gpt-3.5-turbo',
 'response_01': '"New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women\'sHealth"',
 'response_02': 'New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness.

In [105]:
chatbot_dict[4.2]['text1_prompt00'].qna

{'article_title': 'Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women ',
 'system_role': 'You are an expert at science communication.',
 'prompt': 'Take the key points and numerical descriptors to Summarize for a LinkedIn post.',
 'model': 'gpt-3.5-turbo',
 'response_01': '"New study finds that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. In a sample of 14 eumenorrheic women, 24 hours of fluid restriction reduced pain tolerance by 34±46 seconds and increased pain intensity and unpleasantness. Acute water ingestion did not reverse the negative effects of hypohydration on pain sensitivity. #PainResearch #Hydration #Women\'sHealth"',
 'response_02': 'New research shows that mild hypohydration increases pain sensitivity in women, regardless of menstrual phase. A study of 14 eumenorrheic women found that fluid restriction for 24 hours reduced pain tolerance by 34 seconds and increased pain intensity and unpleasantness.

# *End of Page*